In [17]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

import pickle

In [5]:
churn_df = pd.read_csv("../data/task_data_training.csv")


In [6]:
churn_df.head()

,value_number_of_active_months,revenue,value_days_to_purchase,action_create_project,value_transactions_number,ws_users_activated,action_export_report,action_create_invoice,value_regular_seats,action_project_budget,action_time_entries_via_tracker,action_screenshots,churned_status
0,0,184.925,2,5,6,3,8,0,3,0,0,0.0,No
1,3,395.122,98,3,12,2,3,0,3,9,0,1.0,No
2,1,25.974,0,0,2,1,0,0,1,0,0,1.0,Yes
3,1,406.068,53,3,12,3,0,0,3,0,0,1.0,No
4,2,25.974,1,5,2,1,8,6,1,0,0,0.0,No


In [7]:
X = churn_df.drop(['churned_status'], axis=1)
y = churn_df['churned_status']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
# Define the hyperparameter grid to search
param_grid = {
    'C': [0.1, 1, 10],                # Regularization parameter
    'kernel': ['linear', 'rbf', 'sigmoid'],      # Kernel type
    'gamma': ['scale', 'auto', 0.1],  # Kernel coefficient (only for 'rbf' kernel)
}

# Create an SVM classifier
svm_classifier = SVC(random_state=42)

# Create a GridSearchCV object to perform hyperparameter tuning
grid_search = GridSearchCV(estimator=svm_classifier, param_grid=param_grid, scoring='balanced_accuracy', cv=5)

# Fit the grid search to your data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters from the grid search
best_params = grid_search.best_params_

# Use the best hyperparameters to create a new SVM classifier
best_svm_classifier = SVC(random_state=42, **best_params)

# Fit the new classifier to the training data
best_svm_classifier.fit(X_train, y_train)

# Make predictions on the test data using the best model
y_pred_svm = best_svm_classifier.predict(X_test)

# Evaluate the best SVM model
accuracy_svm = accuracy_score(y_test, y_pred_svm)
confusion_svm = confusion_matrix(y_test, y_pred_svm)
classification_rep_svm = classification_report(y_test, y_pred_svm)

print("Best SVM Model after Hyperparameter Tuning:")
print(f"Best Hyperparameters: {best_params}")
print(f"Accuracy: {accuracy_svm}")
print(f"Confusion Matrix:\n{confusion_svm}")
print(f"Classification Report:\n{classification_rep_svm}")

Best SVM Model after Hyperparameter Tuning:
Best Hyperparameters: {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
Accuracy: 0.7288629737609329
Confusion Matrix:
[[173  47]
 [ 46  77]]
Classification Report:
              precision    recall  f1-score   support

          No       0.79      0.79      0.79       220
         Yes       0.62      0.63      0.62       123

    accuracy                           0.73       343
   macro avg       0.71      0.71      0.71       343
weighted avg       0.73      0.73      0.73       343



As we see here, our traditional fine tuning didn't work because we still have a lot of false positives and a lot of false negatives.  So will try now with over/undersampling techniques.

## Over/undersampling methods
#### Undersampling

In [9]:
# Instantiate the RandomUnderSampler
rus = RandomUnderSampler(random_state=42)

# Fit and transform the training data
X_undersampled_train, y_undersampled_train = rus.fit_resample(X_train, y_train)

svm_classifier = SVC(kernel='linear', gamma='scale', C=1, random_state=42) 
svm_classifier.fit(X_undersampled_train, y_undersampled_train)
y_pred = svm_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{confusion}")
print(f"Classification Report:\n{classification_rep}")

Accuracy: 0.6647230320699709
Confusion Matrix:
[[128  92]
 [ 23 100]]
Classification Report:
              precision    recall  f1-score   support

          No       0.85      0.58      0.69       220
         Yes       0.52      0.81      0.63       123

    accuracy                           0.66       343
   macro avg       0.68      0.70      0.66       343
weighted avg       0.73      0.66      0.67       343



#### Oversampling

In [10]:
# Instantiate the RandomOverSampler
ros = RandomOverSampler(random_state=42)

# Fit and transform the training data
X_oversampled_train, y_oversampled_train = ros.fit_resample(X_train, y_train)

svm_classifier = SVC(kernel='linear', gamma='scale', C=1, random_state=42) 
svm_classifier.fit(X_oversampled_train, y_oversampled_train)
y_pred = svm_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{confusion}")
print(f"Classification Report:\n{classification_rep}")

Accuracy: 0.6676384839650146
Confusion Matrix:
[[129  91]
 [ 23 100]]
Classification Report:
              precision    recall  f1-score   support

          No       0.85      0.59      0.69       220
         Yes       0.52      0.81      0.64       123

    accuracy                           0.67       343
   macro avg       0.69      0.70      0.67       343
weighted avg       0.73      0.67      0.67       343



#### SMOTE

In [16]:
# Instantiate the SMOTE sampler
smote = SMOTE(random_state=42, sampling_strategy="not minority")

# Fit and transform the training data
X_oversampled_train, y_oversampled_train = smote.fit_resample(X_train, y_train)

svm_classifier_smote = SVC(kernel='linear', gamma='scale', C=1, random_state=42) 
svm_classifier_smote.fit(X_oversampled_train, y_oversampled_train)
y_pred = svm_classifier_smote.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{confusion}")
print(f"Classification Report:\n{classification_rep}")

Accuracy: 0.7288629737609329
Confusion Matrix:
[[173  47]
 [ 46  77]]
Classification Report:
              precision    recall  f1-score   support

          No       0.79      0.79      0.79       220
         Yes       0.62      0.63      0.62       123

    accuracy                           0.73       343
   macro avg       0.71      0.71      0.71       343
weighted avg       0.73      0.73      0.73       343



In [15]:
# Instantiate the SMOTE sampler
smote = SMOTE(random_state=42, sampling_strategy="not majority")

# Fit and transform the training data
X_oversampled_train, y_oversampled_train = smote.fit_resample(X_train, y_train)

svm_classifier = SVC(kernel='linear', gamma='scale', C=1, random_state=42) 
svm_classifier.fit(X_oversampled_train, y_oversampled_train)
y_pred = svm_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{confusion}")
print(f"Classification Report:\n{classification_rep}")

Accuracy: 0.6880466472303207
Confusion Matrix:
[[138  82]
 [ 25  98]]
Classification Report:
              precision    recall  f1-score   support

          No       0.85      0.63      0.72       220
         Yes       0.54      0.80      0.65       123

    accuracy                           0.69       343
   macro avg       0.70      0.71      0.68       343
weighted avg       0.74      0.69      0.69       343



What we have concluded is that we have either 2 cases of confusion matrix:
\begin{bmatrix}
138 & 82\\
25 & 98
\end{bmatrix}

and the second one

\begin{bmatrix}
173 & 47\\
46 & 77
\end{bmatrix}

The first one is predicting classes fairly correctly, but tells you a lot of times that someone is will be a churn customer (82 times) even though they aren't. And the worst case (false negatives), or in other words - the case where we think someone won't become churn customer and they actually will is still relatively very small. (hence the `Yes recall` is 0.8).


On the other hand second one is really balanced; almost the same amount of FN i FP. Only worryinh thing here is that we have a lot more of false negatives (better overall accuracy, but `Yes recall` is worse compared to the first model).

But our tasks suggest that we should stick to having more balanced model, so we will stick with the second option.


In [19]:
with open('../artifacts/churn_model.pkl', 'wb') as model_file:
    pickle.dump(svm_classifier_smote, model_file)

In [21]:
with open('../artifacts/churn_model_features.pkl', 'wb') as model_file:
    feature_names = list(svm_classifier_smote.feature_names_in_)
    pickle.dump(feature_names, model_file)
